In [ ]:
#TODO:
#Add logger & convert print statements to logging
#add comments

In [39]:
import json
import re

In [5]:
import pandas as pd
pd.options.display.max_rows = 4000

In [6]:
f= open("./archive/quarterly_salary_discussion_jun_2023.json", 'r')
json_deserialized = json.load(f)

In [7]:
print(len(json_deserialized))

2


In [9]:
df = pd.json_normalize(json_deserialized[1]["data"], record_path=["children"], max_level=1)

In [10]:
print(df.head())

  kind data.author_flair_background_color data.subreddit_id  \
0   t1                               None          t5_36en4   
1   t1                               None          t5_36en4   
2   t1                               None          t5_36en4   
3   t1                               None          t5_36en4   
4   t1                               None          t5_36en4   

   data.approved_at_utc  data.author_is_blocked data.comment_type  \
0                   NaN                   False              None   
1                   NaN                   False              None   
2                   NaN                   False              None   
3                   NaN                   False              None   
4                   NaN                   False              None   

  data.awarders data.mod_reason_by data.banned_by data.author_flair_type  ...  \
0            []               None           None                   text  ...   
1            []               None          

In [11]:
print(df.columns)

Index(['kind', 'data.author_flair_background_color', 'data.subreddit_id',
       'data.approved_at_utc', 'data.author_is_blocked', 'data.comment_type',
       'data.awarders', 'data.mod_reason_by', 'data.banned_by',
       'data.author_flair_type', 'data.total_awards_received',
       'data.subreddit', 'data.removed', 'data.author_flair_template_id',
       'data.likes', 'data.replies', 'data.user_reports', 'data.saved',
       'data.id', 'data.banned_at_utc', 'data.mod_reason_title', 'data.gilded',
       'data.archived', 'data.collapsed_reason_code', 'data.no_follow',
       'data.spam', 'data.can_mod_post', 'data.send_replies', 'data.parent_id',
       'data.score', 'data.author_fullname', 'data.removal_reason',
       'data.approved_by', 'data.mod_note', 'data.all_awardings',
       'data.collapsed', 'data.body', 'data.edited', 'data.top_awarded_type',
       'data.author_flair_css_class', 'data.name', 'data.is_submitter',
       'data.downs', 'data.author_flair_richtext', 'data.au

In [12]:
print(df.shape)

(116, 75)


In [69]:
#select columns of interest
transformed_df = df[["data.id","data.author_fullname","data.created","data.body","data.permalink"]]

In [70]:
#Convert UTC dates
transformed_df["data.created"] = pd.to_datetime(df["data.created"],unit='s')

/var/folders/pr/pffq2swj3wjdp25_j3mhffgh0000gn/T/ipykernel_92815/884217166.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  transformed_df["data.created"] = pd.to_datetime(df["data.created"],unit='s')


In [71]:
#Convert subreddit names to URLs
transformed_df["data.permalink"] = 'https://reddit.com' + df["data.permalink"]

/var/folders/pr/pffq2swj3wjdp25_j3mhffgh0000gn/T/ipykernel_92815/3354919982.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  transformed_df["data.permalink"] = 'https://reddit.com' + df["data.permalink"]


In [72]:
#replace double newline, carriage return and tab characters
transformed_df["data.body"] = transformed_df["data.body"].str.replace("\n\n","\n")
transformed_df["data.body"] = transformed_df["data.body"].str.replace("\r","")
transformed_df["data.body"] = transformed_df["data.body"].str.replace("\t","")

/var/folders/pr/pffq2swj3wjdp25_j3mhffgh0000gn/T/ipykernel_92815/2853373037.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  transformed_df["data.body"] = transformed_df["data.body"].str.replace("\n\n","\n")
/var/folders/pr/pffq2swj3wjdp25_j3mhffgh0000gn/T/ipykernel_92815/2853373037.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  transformed_df["data.body"] = transformed_df["data.body"].str.replace("\r","")
/var/folders/pr/pffq2swj3wjdp25_j3mhffgh0000gn/T/ipykernel_92815/2853373037.py:4: SettingWithC

In [75]:
#Split on newline character and save as new df
transformed_df_copy = transformed_df
transformed_df_body = transformed_df["data.body"].str.split('\n',expand=True)

0   \
0                               1. Staff Data Engineer   
1                  1. Title: Senior Analytics Engineer   
2                              1. Title: Data Engineer   
3                              1. Database Engineer II   
4                      1. Title : Senior Data Engineer   
5    1. Current Title: Senior Machine Learning Engi...   
6                          1. Data Engineering Manager   
7                         1) Title: Analytics Engineer   
8                         1. Title: Data Engineer II     
9                        1. title : BI + ETL developer   
10                             1. Title: Data Engineer   
11                             1. Senior Data Engineer   
12                                   1.⁠ Data Engineer   
13                                    1. Data engineer   
14                                    1. Data Engineer   
15                                 1. Sr Data Engineer   
16                            1. Title: Data Engineer    
17                                    1. Current title   
18   1. Title: Analytics Engineer (bit of everythin...   
19                             1. Title: Data Engineer   
20                  1. Data Engineering and BI Manager   
21                             1. Title: Data Engineer   
22                                    1. Everything ?    
23   1.⁠Senior “enterprise technology analyst” (dat...   
24                     1. Title: senior sales engineer   
25                                   1. Data Engineer.   
26                      1. Title: Senior Data Engineer   
27                                  1. Data Engineer     
28                                     1.Data Engineer   
29                                1. Current title: DE   
30                             1.⁠Title: Data Engineer   
31                                           [deleted]   
32                    1. SENIOR TECHNICAL DATA ANALYST   
33                      1. Title: Senior Data Engineer   
34                        Title: senior data engineer    
35                             1. Senior Data Engineer   
36                        1. Title: Lead Data Engineer   
37                            1. Senior Data Engineer    
38                            1. Analytics Engineer II   
39                                    1. Data engineer   
40   1. Title: Senior Data Engineer (More like Data...   
41                             1. Staff Data Engineer    
42                               1. Lead Data Engineer   
43                                                 ```   
44               1. Data Engineer - operations process   
45                                   1. Data engineer    
46      1. Title: Associate Director, Data Engineering   
47                                   1: Data Engineer    
48                                             Awesome   
49                                    1. Data engineer   
50                                                       
51                 1. Senior Data Engineer (Tech Lead)   
52   Not a lot of samples from Canada. Hoping to ge...   
53   I just got a couple job offers and wanted to s...   
54                                           [deleted]   
55                                    1.⁠Data Engineer   
56                             Title: Data Engineer II   
57               1. Title: Data Engineer - market data   
58                                Title: Data Engineer   
59                      1. Title: Senior Data Engineer   
60                                  1. Data Consultant   
61                       1. Title: Azure Data Engineer   
62                               1. Lead Data Engineer   
63                                    1. Data Engineer   
64                              1. Azure Data Engineer   
65                           Title: Data Engineer II     
66   I was laid off in May and recently accepted th...   
67                             1. Title: Data Engineer   
68                             1. Title: Data Engi

In [76]:
#drop extra column
#transformed_df = transformed_df.drop(labels='data.body', axis=1)
#print(transformed_df.shape)


0

In [ ]:
######### EXTRACT JOB TITLE

In [77]:
#standardize title column name
transformed_df_body.rename(columns={transformed_df_body.columns[0]: 'data.title'}, inplace=True)


'data.title'

In [79]:
#remove rows that don't start with any of the following: 1., 1), 1:, Title, Current title, DE, AE, DA, ETL Dev
patterns = ["1.", "1)", "1:", "1-", "Title", "Current Title", "Data Engineer", "Data Analyst", "Analytics Engineer", "ETL Developer"]
filter_condition = ~transformed_df_body["data.title"].str.lower().str.startswith(tuple(pattern.lower() for pattern in patterns))
transformed_df_body.loc[filter_condition] = ""

# transformed_filtered=transformed_df_body.loc[filter_condition]
# # transformed_filtered.to_csv(path_or_buf="./archive/test_filtered.csv", )
# print(transformed_filtered.shape)

In [81]:
#remove unnecessary strings like "1.", etc. from the beginning of the job title column values
transformed_df_body["data.title"] = transformed_df_body["data.title"].str.replace("^[ ]?1[.:)][ ]?","",regex=True)
transformed_df_body["data.title"] = transformed_df_body["data.title"].str.lower().str.replace("^[ ]?(current)?[ ]?(title)?[ ]?[:]?[ ]","",regex=True)
#transformed_filtered["data.title"] = transformed_filtered["data.title"].str.replace("⁠[ ]?title[:]?[ ]?","", regex=True)
#transformed_filtered.to_csv(path_or_buf="./archive/test_filtered.csv", )

In [83]:
#correct some edge cases
transformed_df_body["data.title"] = transformed_df_body["data.title"].str.replace(".*title:?\s?","", regex=True)
#strip white spaces
transformed_df_body["data.title"] = transformed_df_body["data.title"].str.strip()
transformed_df_body["data.title"] = transformed_df_body["data.title"].str.replace("^de$","data engineer", regex=True)
transformed_df_body

data.title  \
0                                  staff data engineer   
1                            senior analytics engineer   
2                                        data engineer   
3                                 database engineer ii   
4                                 senior data engineer   
5                     senior machine learning engineer   
6                             data engineering manager   
7                                   analytics engineer   
8                                     data engineer ii   
9                                   bi + etl developer   
10                                       data engineer   
11                                senior data engineer   
12                                     ⁠ data engineer   
13                                       data engineer   
14                                       data engineer   
15                                    sr data engineer   
16                                       data engineer   
17                                                       
18   analytics engineer (bit of everything tbh, jus...   
19                                       data engineer   
20                     data engineering and bi manager   
21                                       data engineer   
22                                        everything ?   
23   ⁠senior “enterprise technology analyst” (data ...   
24                               senior sales engineer   
25                                      data engineer.   
26                                senior data engineer   
27                                       data engineer   
28                                       data engineer   
29                                       data engineer   
30                                       data engineer   
31                                                       
32                       senior technical data analyst   
33                                senior data engineer   
34                                senior data engineer   
35                                senior data engineer   
36                                  lead data engineer   
37                                senior data engineer   
38                               analytics engineer ii   
39                                       data engineer   
40   senior data engineer (more like data architect...   
41                                 staff data engineer   
42                                  lead data engineer   
43                                                       
44                  data engineer - operations process   
45                                       data engineer   
46                associate director, data engineering   
47                                       data engineer   
48                                                       
49                                       data engineer   
50                                                       
51                    senior data engineer (tech lead)   
52                                                       
53                                                       
54                                                       
55                                      ⁠data engineer   
56                                    data engineer ii   
57                         data engineer - market data   
58                                       data engineer   
59                                senior data engineer   
60                                     data consultant   
61                                 azure data engineer   
62                                  lead data engineer   
63                                       data engineer   
64                                 azure data engineer   
65                                    data engineer ii   
66                                                       
67                                       data engineer   
68                                       d

In [93]:
#Attach title column to original df
transformed_df2 = pd.concat([transformed_df, transformed_df_body['data.title']], axis=1)
